# Train a neural network for binary volumetric segmentation

In this notebook, we will use Nobrainer to train a model for brain extraction. Brain extraction is a common step in processing neuroimaging data. It is a voxel-wise, binary classification task, where each voxel is classified as brain or not brain. Incidentally, the name for the Nobrainer framework comes from creating models for brain extraction.

In the following cells, we will:

1. Get sample T1-weighted MR scans as features and FreeSurfer segmentations as labels.
    - We will binarize the FreeSurfer to get a precise brainmask.
2. Convert the data to TFRecords format.
3. Create two Datasets of the features and labels.
    - One dataset will be for training and the other will be for evaluation.
4. Instantiate a 3D convolutional neural network.
5. Choose a loss function and metrics to use.
6. Train on part of the data.
7. Evaluate on the rest of the data.

## Google Colaboratory

If you are using Colab, please switch your runtime to GPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select GPU under `Hardware accelerator`. A GPU greatly speeds up training.

In [ ]:
!pip install --no-cache-dir nilearn nobrainer

In [ ]:
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaluation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

Here is an example of one training pair, with the brainmask overlaid on the anatomical image.

In [ ]:
import matplotlib.pyplot as plt
from nilearn import plotting
fig = plt.figure(figsize=(12, 6))
plotting.plot_roi(train_paths[0][1], bg_img=train_paths[0][0], alpha=0.4, vmin=0, vmax=1, figure=fig)

# Convert medical images to TFRecords

Remember how many full volumes are in the TFRecords files. This will be necessary to know how many steps are in on training epoch. The default training method needs to know this number, because Datasets don't always know how many items they contain.

In [ ]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

In [ ]:
!mkdir -p data

In [ ]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

In [ ]:
!ls data

# Create Datasets

In [ ]:
n_classes = 1
batch_size = 2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)
n_epochs = None
augment = False
shuffle_buffer_size = 10
num_parallel_calls = 2

In [ ]:
dataset_train = nobrainer.dataset.get_dataset(
    file_pattern='data/data-train_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=n_epochs,
    augment=augment,
    shuffle_buffer_size=shuffle_buffer_size,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern='data/data-evaluate_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=1,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=1,
)

In [ ]:
dataset_train

In [ ]:
dataset_evaluate

# Instantiate a neural network

In [ ]:
model = nobrainer.models.unet(
    n_classes=n_classes, 
    input_shape=(*block_shape, 1),
    batchnorm=True,
)

# A different model can be specified as below.

# model = nobrainer.models.meshnet(
#     n_classes=n_classes, 
#     input_shape=(*block_shape, 1),
# )


In [ ]:
model.summary()

# Choose a loss function and metrics

We have many choices of loss functions for binary segmentation. One can choose from binary crossentropy, Dice, Jaccard, Tversky, and many other loss functions.

In [ ]:
import tensorflow as tf

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

model.compile(
    optimizer=optimizer,
    loss=nobrainer.losses.dice,
    metrics=[nobrainer.metrics.dice, nobrainer.metrics.jaccard],
)

# A different loss function can be specified as below.

# model.compile(
#     optimizer=optimizer, 
#     loss='binary_crossentropy',
# )

# model.compile(
#     optimizer=optimizer, 
#     loss=nobrainer.losses.dice,
# )

# model.compile(
#     optimizer=optimizer, 
#     loss=nobrainer.losses.jaccard,
# )

# model.compile(
#     optimizer=optimizer, 
#     loss=nobrainer.losses.tversky,
# )

# Train and evaluate model

$$
steps = \frac{nBlocks}{volume} * \frac{nVolumes}{batchSize}
$$

In [ ]:
steps_per_epoch = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

steps_per_epoch

In [ ]:
validation_steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(evaluate_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

validation_steps

The following step takes about 10-12 mins with the GPU backend for 5 epochs. You can increase the number of epochs to lower the loss and improve the accuracy of the model.




In [ ]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch, 
    validation_data=dataset_evaluate, 
    validation_steps=validation_steps)

# Predict medical images without TFRecords

In [ ]:
from nobrainer.volume import standardize
import nibabel as nib

image_path = evaluate_paths[0][0]
out = nobrainer.prediction.predict_from_filepath(image_path, 
                                           model,
                                           block_shape = block_shape,
                                           batch_size = batch_size,
                                           normalizer = standardize,
                                             )
out.shape

In [ ]:
fig = plt.figure(figsize=(12, 6))
plotting.plot_roi(out, bg_img=image_path, alpha=0.4, figure=fig)